In [1]:
from torch.autograd import Variable
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.optim as optim
import random



# Generate the communtity list

def make_sents(sp_matrix):
    sent = []
    unfort_list = []
    ii =0
    for i in range(sp_matrix.shape[0]):  

        pd_table_sub = sp_matrix.iloc[i,:]>0
        c_v = sp_matrix.iloc[i,:][sp_matrix.iloc[i,:]>0]
        if len(c_v)>1:
            sent.append(c_v)
        else:
            unfort_list.append(ii)
        ii += 1
        print('\r  %s...'%ii,end = '')
    return(sent)



# The main function for ecological baesd glove
def glove_dic(sent,embed_size,vocab,wf,num_epochs = 5,glove_abuna = True):
    # Set parameters
    xmax = 2
    alpha = 0.75
    batch_size = 100
    l_rate = 0.001
    num_epochs = num_epochs

    # Create vocabulary and word lists
    word_list = sent
    w_list_size = len(word_list)
    vocab_size = len(vocab)

    # Create word to index mapping
    w_to_i = {word: ind for ind, word in enumerate(vocab)}
    comat = np.zeros((vocab_size, vocab_size))
    if glove_abuna == True:
        # Construct co-occurence matrix
        for i_init in range(w_list_size):
            cc = sent[i_init]
            for i in range(len(cc)):
                for j in range(len(cc)):
                    if i != j:
                        ind = w_to_i[cc.index[i]]
                        lind = w_to_i[cc.index[j]]
                        comat[ind, lind] += cc.values[i]
                        comat[lind,ind] += cc.values[j]
    # Non-zero co-occurrences
    else:
        for i_init in range(w_list_size):
            cc = sent[i_init]
            for i in range(len(cc)):
                for j in range(len(cc)):
                    if i != j:
                        ind = w_to_i[cc.index[i]]
                        lind = w_to_i[cc.index[j]]
                        comat[ind, lind] += 1
                        comat[lind,ind] += 1

    coocs = np.transpose(np.nonzero(comat))
    # Weight function
    # Set up word vectors and biases
    l_embed, r_embed = [
	[Variable(torch.from_numpy(np.random.normal(0, 0.01, (embed_size, 1))),
		requires_grad = True) for j in range(vocab_size)] for i in range(2)]
    l_biases, r_biases = [
        [Variable(torch.from_numpy(np.random.normal(0, 0.01, 1)), 
            requires_grad = True) for j in range(vocab_size)] for i in range(2)]

    # Set up optimizer
    optimizer = optim.Adam(l_embed + r_embed + l_biases + r_biases, lr = l_rate)

    # Batch sampling function
    def gen_batch():	
        sample = np.random.choice(np.arange(len(coocs)), size=batch_size, replace=False)
        l_vecs, r_vecs, covals, l_v_bias, r_v_bias = [], [], [], [], []
        for chosen in sample:
            ind = tuple(coocs[chosen])
            l_vecs.append(l_embed[ind[0]])
            r_vecs.append(r_embed[ind[1]])
            covals.append(comat[ind])
            l_v_bias.append(l_biases[ind[0]])
            r_v_bias.append(r_biases[ind[1]])
        return l_vecs, r_vecs, covals, l_v_bias, r_v_bias

    # Train model
    for epoch in range(num_epochs):
        num_batches = int(w_list_size/batch_size)
        avg_loss = 0.0
        for batch in range(num_batches):
            optimizer.zero_grad()
            l_vecs, r_vecs, covals, l_v_bias, r_v_bias = gen_batch()
            # For pytorch v2 use, .view(-1) in torch.dot here. Otherwise, no need to use .view(-1).
            loss = sum([torch.mul((torch.dot(l_vecs[i].view(-1), r_vecs[i].view(-1)) +
                    l_v_bias[i] + r_v_bias[i] - np.log(covals[i]))**2,
                    wf(covals[i])) for i in range(batch_size)])
            avg_loss += loss.data[0]/num_batches
            loss.backward()
            optimizer.step()
        print("\r Average loss for epoch "+str(epoch+1)+": ", avg_loss.item(),end = '')
    print(' ')
    lem_list =[];rem_list=[];spname_list = []
    for word_ind in range(len(vocab)):
        # Create embedding by summing left and right embeddings
        rem_list.append(l_embed[word_ind].data.numpy())  
        spname_list.append(vocab[word_ind])
    #cc = pd.DataFrame(np.array(lem_list)[:,:,0])
    #cc['Species'] = spname_list
    emb_dic = pd.DataFrame(np.array(rem_list)[:,:,0])
    #emb_dic['Species'] = spname_list
    emb_dic.index = vocab
    
    return(emb_dic)

# Generate the communtity embedding

def make_com_emb(sp_matrix,emb_dic,embed_size):
    emb_sent = []
    for i in range(sp_matrix.shape[0]):  

        pd_table_sub = sp_matrix.iloc[i,:]#>0
        c_v = pd_table_sub[pd_table_sub>0].index#np.unique(pd_table_sub.Species.dropna())
        init_vec = np.zeros([embed_size])
        for ii in c_v:
            #i_sum += 1
            init_vec += np.float32(emb_dic.loc[ii].values)

        emb_sent.append(init_vec/len(c_v)) #/i_sum

    #print('\r  %s.../95104'%i,end = '')
    return(np.array(emb_sent))

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
import lightgbm as lgb

from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,ARDRegression

import argparse
import pprint
#import gensim
from sklearn.model_selection import cross_validate,LeaveOneOut
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV,train_test_split
import pickle
import math
#import random
from sklearn.model_selection import KFold

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,accuracy_score,recall_score
import random
def to_mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)
def to_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
def to_mse(y_true, y_pred):
    return (mean_squared_error(y_true, y_pred))  

In [6]:

# The embedding name for printing importantce indicators
def emb_names(table):
    new_p_list =[]
    for i in range(table.shape[1]):
        new_p_list.append('em'+str(i))
    return(new_p_list)

# Keep the fossile data have the same taxon with normal data set
def trans_to_core(spec,core):
    new_pd = pd.DataFrame(np.zeros([core.shape[0], spec.shape[1]]))
    new_pd.columns = spec.columns
    for i_index,i_names in enumerate(new_pd.columns):
        try:            
            new_pd.iloc[:,i_index] = core[i_names].values
        except:
            continue
    return(new_pd)

import warnings
warnings.filterwarnings("ignore")

def to_pers(df):
    row_sums = df.sum(axis=1)
    # Divide each value in the row by the corresponding row sum and multiply by 100
    df_percentage = df.div(row_sums, axis=0) 
    return(df_percentage)

def gcd_many(s):
    g = 0
    for i in range(len(s)):
        if i == 0:
            g = s[i]
        else:
            g=math.gcd(g,s[i])
    return g


def to_num(matrix_init):
    matrix = matrix_init.copy()
    for i in range(matrix.shape[0]):
        sub_raw = matrix.iloc[i,:].values
        sub_raw_2 = sub_raw[sub_raw>0]
        min_raw  = min(sub_raw_2)
        sub_raw = sub_raw*10000#sub_raw/min_raw  ## 0/min_raw  = 0
        sub_raw = sub_raw/gcd_many(sub_raw[sub_raw>0].astype('int'))
    matrix.iloc[i,:] = sub_raw
    return(matrix)


# The main function for MEMLM
def pre_glove(Biological_abundance_matrix,environmental_values_matrix,test_method = 'loo',no_components = 256, data_name='Test',
              add_tr_env = [], cores = [],core_sp = [],stack = True, glove_abuna = True,random_no = 123,
              recon=False,pass_va = False,save_vali = True,cor_emb = False, emb_method = 'plus',test_set = [],num_epochs = 300):
    random_no = random_no
    # set the flags
    if len(test_set) >0:
        recon= False
        pass_va = False
        cores = []
        
    if emb_method == 'only':
        only_emb = True
        only_abun = False
        print('using the embedding matrix')
    elif emb_method == 'none':
        only_abun = True
        only_emb = False
        print('using the abundance matrix')
    else:
        only_abun = False
        only_emb = False
        print('using the embedding + abundance matrix')
    # Set the random seeds
    random.seed(random_no)
    torch.manual_seed(random_no)
    torch.cuda.manual_seed(random_no)  # If using CUDA
    torch.manual_seed(random_no)

    # Set the random seed for NumPy
    np.random.seed(0)   
        
        
        
    pd_table = Biological_abundance_matrix
    envs = environmental_values_matrix
    #save_vali = save_vali            
    if recon ==False:
        cores = []
    else:
        #print(cores.shape)
        cor_sp = (set(cores.columns)&set(pd_table.columns))
    try:
        (envs.shape[1])
    except:
        envs = pd.DataFrame(envs.T)
    
    #Statistics of species contained in each sample
    print('scale: %s'%pd_table.iloc[0,:].sum())
    xmax = 2 
    alpha = 0.75
    def wf(x):
        return x
    if glove_abuna == True:
        if (pd_table.iloc[0,:].sum() > 1.3):
            pd_table_2 = pd_table
            #if np.min(pd_table_2[pd_table_2>0]).min()>1:
            def wf(x):
                if x < xmax:
                    return (x/xmax)**alpha
                return 1
        else:
            pd_table_2 = to_num(pd_table)

    #pd_table_sub = pd_table.iloc[i,:]*250
    if recon == True:
        if cor_emb == True:
            pd_table_2 = pd_table_2[cor_sp]
    if only_abun == False:
        sent = make_sents(pd_table)
        print('creating the embedding now...')   ###
    
        embed_size = no_components
        vocab = pd_table.columns
        #embed_size = 256
        emb_dic = glove_dic(sent,embed_size,vocab,wf,glove_abuna = glove_abuna,num_epochs = num_epochs)
        
        paragraph_list = make_com_emb(pd_table,emb_dic,embed_size)
        emb_name_list = list(emb_names(paragraph_list))

        #print('Species num(availble in asscoiation): %s' % len(corpus_model.dictionary))
    print('Species num: %s' % pd_table.shape[1])
    print('Samples num: %s' % pd_table.shape[0])
    print(' ')
    print('=================================')
    print('training the embedding ......')
    


    
    if only_emb == True:  
        para_names =  emb_name_list   

    elif only_abun == True:
        para_names = pd_table.columns
        paragraph_list = np.array(pd_table)
    else:
        print('adding the speiese abunadance now...')
        paragraph_list = np.concatenate((np.array(paragraph_list),np.array(pd_table)), axis = 1)    ###
        data_name = '%s_rich'%data_name
        para_names =  emb_name_list + list(pd_table.columns)
        #print([len(para_names),len(list(emb_names(np.array(paragraph_list)))),len(list(emb_names(np.array(paragraph_list))))])    
        #print(paragraph_list.shape)
        
    if  len(add_tr_env) !=0:
        data_name = '%s_added'%data_name
        paragraph_list = np.concatenate((np.array(paragraph_list),np.array(add_tr_env)), axis = 1)
        para_names = emb_name_list + list(add_tr_env.columns)
        #print(paragraph_list.shape)
    #print(len(para_names))

        #print(paragraph_list.shape,cores.shape)
    if pass_va == False :  
        if test_method == 'cv':
            cs=train_data_pre(paragraph_list,envs,data_name=data_name,test_method='cv',para_names = para_names, stack = stack, save_vali = save_vali)
        elif test_method == 'loo':
            cs=train_data_pre(paragraph_list,envs,data_name=data_name,test_method='loo',para_names = para_names, stack = stack, save_vali = save_vali)
        elif test_method == 'diy':
            cs=train_data_pre(paragraph_list,envs,data_name=data_name,test_method='diy',para_names = para_names, stack = stack,test_set=test_set)

        else:
            print('you shall choose one of these validate method:  cv, loo,diy') 
    if len(cores) !=0:
        print('reconstructing the %s'%data_name)
        cores_index = cores.index
        if len(cores.columns) > len(pd_table.columns):      
            cores = cores[cor_sp]
            
        elif len(cores.columns) < len(pd_table.columns):
            cores = trans_to_core(pd_table,cores)
        paragraph_list2 = []
        #Generate embedding for each sample
        cores.columns = pd_table.columns
        if only_abun == False:
            paragraph_list2 = make_com_emb(cores,emb_dic,embed_size)
            if only_emb == False:
                paragraph_list = np.concatenate((np.array(paragraph_list)[:,:no_components],np.array(pd_table)), axis = 1)
                para_names = emb_name_list+list(pd_table.columns)
                cores = np.concatenate((np.array(paragraph_list2),np.array(cores)), axis = 1)    #
            else:
                cores = np.array(paragraph_list2)
        cs = recon_data(paragraph_list,envs,cores,cores_index,pass_va,data_name=data_name,trastacks = stack,test_method=test_method,para_names = para_names)
        
    print('The operation is over. Have a nice day!')
    print('')
    print('')

    return(cs)

def train_data_pre(X,y,data_name,GBR = False,n_estimators = 1000,test_method = 'loo',stack = True,save_vali = False,para_names=[],test_set = []):
    if save_vali == True:
        op_pirnt = 'on'
    else:
        op_pirnt = 'off'
    print('save validation results option: %s'%op_pirnt)
    import warnings
    warnings.filterwarnings("ignore")
    acc_tt_list = []
    try:
        y.shape[1]
        init_name = data_name
        print('has multi environmental values')
        for y_col in range(y.shape[1]):
            pd_acc = pd.DataFrame(np.zeros([y.shape[1]*2,5]))
            data_name = '%s_%s'%(init_name,y.columns[y_col])
            X =np.array(X)
            yy =np.array(y.iloc[:,y_col])
            xx= X[~pd.isnull(yy)]
            print('sub mission:  training %s now...'%data_name)
            if test_method== 'loo':
                acc = train_data_loo(X=xx,y = yy,GBR = False,data_name=data_name,n_estimators = 1000,stack = stack, save_vali = save_vali,test_method=test_method,para_names = para_names)
            elif test_method== 'cv':
                acc = train_data_cv(X=xx,y = yy,GBR = False,data_name=data_name,n_estimators = 1000,stack = stack, save_vali = save_vali,test_method=test_method,para_names = para_names)
            elif test_method== 'diy':
                return(train_data_diy(X=xx,y = yy,test_set = test_set, GBR = False,data_name=data_name,n_estimators = 1000,stack = stack, test_method=test_method,para_names = para_names))

        acc_tt_list.append([data_name,*acc])  
        return(acc_tt_list)
    except:
        if test_method == 'split':
            return(train_data_split_across(X,y,GBR = False,data_name=data_name,n_estimators = 1000,stack = stack,test_method=test_method,para_names = para_names))
        if test_method == 'loo':
            return(train_data_loo(X,y,GBR = False,data_name = data_name,n_estimators = 1000,stack = stack, save_vali = save_vali,test_method=test_method,para_names = para_names))
        elif test_method == 'cv':
            return(train_data_cv(X,y,GBR = False,data_name=data_name,n_estimators = 1000,stack = stack, save_vali = save_vali,test_method=test_method,para_names = para_names))     
        elif test_method== 'diy':
            return(train_data_diy(X=xx,y = yy,test_set = test_set, GBR = False,data_name=data_name,n_estimators = 1000,stack = stack, test_method=test_method,para_names = para_names)
)
def train_data_cv(X,y,data_name,test_method = [],para_names = [],GBR = False,n_estimators = 1000,stack = True,save_vali = False):       
    
    base_list = np.array(range(X.shape[0]))
    x_train_names = para_names
    ran_index = np.random.permutation(base_list)
    y= np.array(y)
    X = np.array(X)
    X = X[ran_index]
    y =y[ran_index]
    y_token = 'multi'
    y_test_l = []
    pre_l = []
    kf = KFold(n_splits=5)

    for train, test in kf.split(base_list):
    
        try:
            x_train, x_test, y_train, y_test = X[train],X[test],y[train],y[test]
        except:
            x_train, x_test, y_train, y_test = X.loc(axis=0)[train],X.loc(axis=0)[test],y.loc(axis=0)[train],y.loc(axis=0)[test]        
        #print(x_train.shape,y_train.shape)
        predict3 = make_predict(x_train, x_test, y_train, y_test,test_method,data_name,x_train_names)
        if stack == True:

            predict3 = to_stack_2(x_train,y_train,predict3)

        y_test_l.append(y_test.reshape(-1,1))
        pre_l.append(predict3)
    
    pd_re = tail(pre_l,y_test_l,predict3,data_name,stack = stack,test_method ='cv',save_vali = save_vali)
    return(pd_re) #[choose_predict(acc)] #,y_test, predict3,rfr



def train_data_diy(X,y,test_set,data_name,test_method = [],para_names = [],GBR = False,n_estimators = 1000,stack = True):       

    x_train_names = para_names
    X =np.array(X)
    y = np.array(y)
    y_token = 'multi'
    print('test',X.shape)

    
    x_train = X
    x_test = y
    y_test = []
    y_train = test_set
    #print('MEASURED     RECON       RICHNESS')

    predict3 = make_predict(x_train, x_test, y_train, y_test,test_method,data_name,x_train_names,print_importa = False)
    if stack == True:
        #print(y_train.shape,x_train.shape,np.array(predict3).shape)

        predict3 = to_stack_2(x_train,y_train,predict3)
        #print('OK')
    return(predict3) #[choose_predict(acc)] #,y_test, predict3,rfr

def make_predict(x_train, x_test, y_train, y_test,data_name , test_method,x_train_names,n_estimators = 1000, GBR = False, print_importa = True):
    
    if len(x_train.shape) == 1:
            x_train = x_train.reshape(-1, 1)
            x_test = x_test.reshape(-1, 1)
    if len(y_test)>0:
        if len(y_test.shape) == 1:
            y_train = y_train.reshape(-1, 1)
            y_test = y_test.reshape(-1, 1)
            y_token = 'single'
    #print([x_train.shape,x_test.shape])
    predict3 = three_models( x_train,y_train,x_test,test_method,data_name, x_train_names, print_importa = True,print_model = False)


    return(predict3)
        
def to_stack(x_train,y_train,predict3):
    rfr2,best_no, quit_lightGBM = mulit_model_stack2(x_train,y_train)
    #print(rfr2,best_no, quit_lightGBM)
    acc3_l = []
    acc3_l.append(np.array(predict3)[0,:])
    acc3_l.append(np.array(predict3)[1,:])
    acc3_l.append(np.array(predict3)[2,:])
    #print(acc3_l)
    #print(predict3)
    if quit_lightGBM == True:
        predict3.append(rfr2.predict((np.array(acc3_l).T)[:,:1]).reshape(-1,1)
                       )
    else:
        predict3.append(rfr2.predict(np.array(acc3_l).T).reshape(-1,1))
    #print(predict3)
    return(predict3)

def to_stack_2(x_train,y_train,predict3):
    rfr2,best_no, quit_lightGBM = mulit_model_stack2(x_train,y_train,test_method = 'cv')
    predict3 = np.array(predict3).T   # (3,8)
    acc3_l = []
    acc3_l.append(predict3[:,0])
    acc3_l.append(predict3[:,1])
    acc3_l.append(predict3[:,2])

    if quit_lightGBM == True:
                predict3 = np.concatenate((predict3,rfr2.predict(np.array(acc3_l).T)[:,:1].reshape(-1,1)
                                          ), axis = 1) 
    else:
        predict3 = np.concatenate((predict3,rfr2.predict(np.array(acc3_l).T).reshape(-1,1)), axis = 1) 
    #print(predict3)
    return(predict3)
    
    
def train_data_loo(X,y,test_method, data_name,para_names,GBR = False,n_estimators = 1000, stack = False,save_vali = False):
    y = np.array(y)
    x_train_names = para_names
    X =np.array(X)
    y_token = 'multi'
    X_no =range(X.shape[0])
    loo = LeaveOneOut()
    y_test_l = []
    pre_l = []
    print('MEASURED     RECON       RICHNESS')

    # Loo cross_validate
    for train, test in loo.split(X_no):

        x_train, x_test, y_train, y_test = X[train],X[test],y[train],y[test]

        #x_train, x_test, y_train, y_test = train_test_split( X,y, test_size=0.50, random_state=33)
        
        predict3 = make_predict(x_train, x_test, y_train, y_test,test_method,data_name,x_train_names)

        if stack == True:
            predict3 = to_stack(x_train,y_train,predict3)

        y_test_l.append(y_test)
        pre_l.append(predict3)
        print("   %.3f      %.3f         %s" %(pre_l[-1][0],y_test_l[-1][0],len(x_test[x_test>0])))
    #acc = []
    pd_re = tail(pre_l,y_test_l,predict3,data_name,stack = stack,test_method = 'loo',save_vali = save_vali)
    
    return(pd_re) #[choose_predict(acc)] #,y_test, predict3,rfr

def tail(pre_l,y_test_l,predict3,data_name,stack = False,test_method = 'loo',save_vali = False):
    if test_method == 'cv':
        predict3 = np.vstack(pre_l)
        #print(predict3.shape)
    else:
        predict3 = np.array(pre_l)
    y_test =  np.vstack(y_test_l).reshape(-1, 1)

    acc_r2 = []
    acc_rmse = []
    #print(y_test.shape,predict3.shape)
    #if test_method == 'cv':
    #predict3 = predict3.T
    for i in range(predict3.shape[1]):

        try:
            acc_r2.append(r2_score(y_test, predict3[:,i].reshape(-1, 1)))
            acc_rmse.append(to_rmse(y_test, predict3[:,i].reshape(-1, 1)))
        except:
            acc_r2.append(r2_score(y_test, predict3[:,i]))
            acc_rmse.append(to_rmse(y_test, predict3[:,i]))
    try:
        acc_r2.append(r2_score(y_test, predict3.mean(1).reshape(-1, 1)))
        acc_rmse.append(to_rmse(y_test, predict3.mean(1).reshape(-1, 1)))
    except:
        acc_r2.append(r2_score(y_test, predict3[:,i].mean(1)))
        acc_rmse.append(to_rmse(y_test, predict3[:,i].mean(1)))
    acc = [acc_r2,acc_rmse]
      
    print("___________result___________")
    print("r2_score")
    if stack == False:
        print("   rfr       etr          lightGBM")
        print("   %.3f     %.3f        %.3f" %(acc[0][0],acc[0][1],acc[0][2]))
        print("RMSEP")

        print("   rfr       etr          lightGBM")
        print("   %.3f     %.3f        %.3f" %(acc[1][0],acc[1][1],acc[1][2]))
        va_cloumns = ["rfr"  ,     "etr",          "lightGBM",  "true"]
    else:

        print("   rfr       etr       lightGBM       stack       mean")
        print("   %.3f     %.3f        %.3f       %.3f       %.3f" %(acc[0][0],acc[0][1],acc[0][2],acc[0][3],acc[0][4]))
        print("RMSEP")

        print("   rfr       etr       lightGBM       stack       mean")
        print("   %.3f     %.3f        %.3f       %.3f       %.3f" %(acc[1][0],acc[1][1],acc[1][2],acc[1][3],acc[1][4]))
        va_cloumns = ["rfr"  ,     "etr",          "lightGBM",   "stack",'mean',"true"]
    ##pickle.dump([predict3,y_test],open('./pkl/cross_val_%s.pkl'%(data_name),"wb"), protocol=3)
    #print(predict3.shape,y_test.shape)


    if save_vali == True:
        try:
            va_results = pd.DataFrame(np.concatenate((predict3[:,:],y_test),axis = 1))
        except:
            va_results = pd.DataFrame(np.concatenate((predict3[:,:,0],y_test),axis = 1))
        va_results.columns = va_cloumns[:-1]
        va_results.to_csv('vadata_%s_%s.csv'%(test_method,data_name))
    #pickle.dump([predict3,y_test],open('./pkl/vali_%s_%s.pkl'%(test_method,data_name),"wb"), protocol=3)
    #r2_score(y_test.reshape((-1, 1)), predict3[-1].reshape((-1, 1)))
    #mo.score(y_test.reshape((-1, 1)), predict3[0]
    model_num = predict3.shape[1]
    pd_re = pd.DataFrame(acc)
    pd_re.columns =  ['rfr','etr','lightGBM','stack','mean']#[:model_num]
    pd_re.index = ['r2_score','rmsep']
    pd_re.to_csv('varesult_%s_%s.csv'%(test_method,data_name))   ######
    return(pd_re)



def recon_data(paragraph_list,envs,cores,cores_index,pass_va,para_names,test_method, data_name,GBR =False,trastacks = True):
    ## starting recon data process!
    y_train = envs
    x_train_names = para_names
    x_train = np.array(paragraph_list)
    x_test = cores
    predict3 = []
    
    if len(y_train.shape) == 1:
        y_train = y_train.reshape(-1, 1)
        y_token = 'single'
    if pass_va == False:
        print_importa = False
    else:
        print_importa = True
    predict3 = three_models( x_train,y_train,x_test,test_method,data_name,x_train_names, print_importa = print_importa,print_model = False)
                 
    ## starting stack!!
    if trastacks == True:
        model_num = 4
        rfr2,best_no, quit_lightGBM = mulit_model_stack2(x_train,y_train)

        acc3_l = []
        acc3_l.append(np.array(predict3)[0,:])
        acc3_l.append(np.array(predict3)[1,:])
        acc3_l.append(np.array(predict3)[2,:])        #print(np.array(acc3_l).shape)
        if quit_lightGBM == True:
            predict3.append(rfr2.predict((np.array(acc3_l).T)[:,:1]).reshape(-1,1))
        else:
            predict3.append(rfr2.predict(np.array(acc3_l).T).reshape(-1,1)
                           )
    else:
        model_num = 3
        print('without stack process...')
    model_name_list = []
    for i in range(model_num):
        if i == 3:
            i = -1
        model_name = ['rfr','etr','lightGBM','stack'][i]
        pd.DataFrame(predict3).T
        #pd.DataFrame(predict3[i]).to_csv('./recon/%s_%s.csv'%(data_name,model_name))
        print('created the %s_%s model output'%(data_name,model_name))
        model_name_list.append(model_name)
    ## reconstructions has been done
    if data_name != 'test':
        t_pd = pd.DataFrame(predict3).T
        t_pd.columns = model_name_list
        if model_name_list[-1] =='stack':
            #t_pd['stack'] =  t_pd['stack'].apply(lambda x: x.replace('[','').replace(']','')) 
            t_pd['stack'] =  t_pd['stack'].apply(lambda x: x[0]) 
            t_pd['stack'] = t_pd['stack'].astype(float)
        t_pd.index = cores_index
        t_pd.to_csv('%s_core.csv'%(data_name))
    return(t_pd)



def mulit_model_stack2(x_train,y_train,GBR = False,test_method = 'loo'):
    
    test4 = []
    predict4 = []
    predict5_l = []
    best_no = []
    acc_list = []
    acc_list2 = []

    x_train=np.array(x_train)  #
    y_train = np.array(y_train)
    kf = KFold(n_splits=5)

    
    base_list = np.array(range(y_train.shape[0]))
    ran_index = np.random.permutation(base_list)
    x_train = x_train[ran_index]
    y_train =y_train[ran_index]
    pre_l = []
    for train, test in kf.split(ran_index):
        predict5 =[]
        
        try:
            x_train2, x_test2, y_train2, y_test2 = x_train[train],x_train[test],y_train[train],y_train[test]
        except:
            x_train2, x_test2, y_train2, y_test2 = x_train.loc(axis=0)[train],x_train.loc(axis=0)[test],y_train.loc(axis=0)[train],y_train.loc(axis=0)[test]
            
        predict3 = three_models( x_train2,y_train2,x_test2, print_importa = False,print_model = False)
        #print('pred',np.array(predict3).shape)
        test4.append(y_test2.reshape(-1,1))
        predict4.append(np.array(predict3).T)
        # predict3 (3, 4)
        use_inf = True
        if test_method == 'loo':
                predict5.append(np.array(predict3)[0,:])
                predict5.append(np.array(predict3)[1,:])
                predict5.append(np.array(predict3)[2,:])        
        else:
            if use_inf == True:
                predict5.append(np.array(predict3)[0,:])
                predict5.append(np.array(predict3)[1,:])
                predict5.append(np.array(predict3)[2,:])
            else:
                
                predict5.append((np.array(predict3)[0,:]+np.array(predict3)[1,:]).T/2)
                predict5.append((np.array(predict3)[0,:]+np.array(predict3)[2,:]).T/2)
                predict5.append((np.array(predict3)[1,:]+np.array(predict3)[2,:]).T/2)
                predict5.append((np.array(predict3)[1,:]+np.array(predict3)[2,:]+np.array(predict3)[0,:]).T/3)
       
        predict5_l.append(np.array(predict5).T)

        acc2 = []
        predict5= np.array(predict5).T

        
    rfr2 = LinearRegression() #ARDRegression() LinearRegression() 
    #n_estimators = 1000, min_sample_leaf10)
    predict5_lt = np.vstack(predict5_l)
    #print('pred2',predict5_lt.shape)

    test_lt = np.vstack(test4)

    quit_lightGBM = False

    rfr2.fit(np.vstack(predict5_lt),test_lt) 
    return(rfr2,best_no,quit_lightGBM)

def three_models(x_train,y_train,x_test,test_method =[],data_name=[],x_train_names=[],GBR =False, print_importa = True,print_model = False,n_estimators = 1000,random_no = 123):
    from numpy import random
    #print(x_train.shape,'three_models')
    predict3 = []
    rfr = RandomForestRegressor(n_estimators = 1000,n_jobs=-2, random_state = random.seed(random_no))#n_estimators , min_sample_leaf10)
    # training
    rfr.fit(x_train,y_train) #.reshape((-1, 1)),
    # predict and save forecast result
    predict3.append(rfr.predict(x_test)) # .reshape((-1, 1))


    etr = ExtraTreesRegressor(n_estimators = n_estimators,n_jobs=-2, random_state = random.seed(random_no))
    etr.fit(x_train,y_train) #.reshape((-1, 1)),
    predict3.append(etr.predict(x_test))
    if GBR == True:
        gbr = GradientBoostingRegressor(n_estimators = n_estimators,n_jobs=-2, random_state = random.seed(random_no))
        gbr.fit(x_train,y_train) #.reshape((-1, 1)),
        gbr_y_predict = gbr.predict(x_test)
        predict3.append(gbr.predict(x_test))
        predict3 = np.array(predict3)
        print('the 3rd model is GBR')

        imp_table.columns = ['rfr','etr']
    else:
        gbm = LGBMRegressor(objective='regression', num_leaves=31, learning_rate=0.05, n_estimators=1000,n_jobs=-2,random_state = random.seed(random_no),verbose = -1)

        gbm.fit(x_train,y_train) #.reshape((-1, 1)),
        predict3.append(gbm.predict(x_test))
        imp_table = pd.DataFrame([rfr.feature_importances_,etr.feature_importances_,gbm.feature_importances_]).T
        imp_table.columns = ['rfr','etr','LightGBM']
    if print_importa == True:
        

        if  len(x_train_names) >0:
            imp_table.index = x_train_names
        imp_table = imp_table.sort_values(['rfr'],ascending = False)
        imp_table.to_csv('importances_%s_%s.csv'%(test_method,data_name))
    if print_model == False:
        return(predict3)
    else:
        return(predict3,rfr,etr,model3)


In [8]:
no_components = 256
num_epochs = 1000
date = '0101'
name = 'rlgh'
swap_sp= pd.read_csv('swap_spec.csv',index_col = 0)  #/100
swap_ph= pd.read_csv('swap_ph.csv',index_col = 0)
rlgh_spec= pd.read_csv('rlgh_spec.csv',index_col = 0)
_ = pre_glove(swap_sp/100,swap_ph,cores = rlgh_spec/100,emb_method = 'plus',stack = True,no_components=no_components,data_name='%s_ae%s_%s'%(name,no_components,date),test_method = 'cv',num_epochs= 1000,glove_abuna = True,recon=True,pass_va = True)
_ = pre_glove(swap_sp/100,swap_ph,cores = rlgh_spec/100,emb_method = 'only',stack = True,no_components=no_components,data_name='%s_e%s_%s'%(name,no_components,date),test_method = 'cv',num_epochs= 1000,glove_abuna = True,recon=True,pass_va = True)
_ = pre_glove(swap_sp/100,swap_ph,cores = rlgh_spec/100,emb_method = 'none',stack = True,no_components=1,data_name='%s_a%s_%s'%(name,no_components,date),test_method = 'cv',glove_abuna = True,recon=True,pass_va = True)


using the embedding + abundance matrix
scale: 0.8427
  167...creating the embedding now...
 Average loss for epoch 1000:  7.211855607312498 
Species num: 277
Samples num: 167
 
training the embedding ......
adding the speiese abunadance now...
reconstructing the rlgh_ae256_0101_rich
created the rlgh_ae256_0101_rich_rfr model output
created the rlgh_ae256_0101_rich_etr model output
created the rlgh_ae256_0101_rich_lightGBM model output
created the rlgh_ae256_0101_rich_stack model output
The operation is over. Have a nice day!


using the embedding matrix
scale: 0.8427
  167...creating the embedding now...
 Average loss for epoch 1000:  7.211855607312498 
Species num: 277
Samples num: 167
 
training the embedding ......
reconstructing the rlgh_e256_0101
created the rlgh_e256_0101_rfr model output
created the rlgh_e256_0101_etr model output
created the rlgh_e256_0101_lightGBM model output
created the rlgh_e256_0101_stack model output
The operation is over. Have a nice day!


using the abu